AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [ ]:
# !pip install autogluon.timeseries

In [2]:
# Importar librerías
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

/home/nespina/Documentos/austral/labo_3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar datasets
df_sellin = pd.read_csv("../data/sell-in.txt", sep="\t")
df_productos = pd.read_csv("../data/tb_productos.txt", sep="\t")

# Leer lista de productos a predecir
with open("../data/product_id_apredecir201912.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

# Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [4]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [5]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [ ]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product.head()

In [7]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [8]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [9]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

In [ ]:
# Filter out unsupported models and convert to dictionary format
supported_models = ['DeepAR', 'PatchTST', 'TemporalFusionTransformer']
hyperparameters_dict = {model: {} for model in supported_models}

predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
)

predictor.fit(
    ts_data, 
    num_val_windows=2, 
    time_limit=60*60, 
    presets=None, 
    # hyperparameters=hyperparameters_dict,
    enable_ensemble=True
)


Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/home/nespina/Documentos/austral/labo_3/src/AutogluonModels/ag-20250719_004328'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #66-Ubuntu SMP PREEMPT_DYNAMIC Fri Jun 13 20:25:30 UTC 2025
CPU Count:          8
GPU Count:          0
Memory Avail:       5.69 GB / 11.37 GB (50.0%)
Disk Space Avail:   107.99 GB / 284.85 GB (37.9%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': {'DeepAR': {},
                     'PatchTST': {},
                     'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': Fa

In [13]:
# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [14]:
# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

Index(['item_id', 'timestamp', 'mean'], dtype='object')


In [15]:
# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


In [16]:
# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_01_07.csv", index=False)
resultado.head()

,product_id,tn
1,20001,1277.140118
3,20002,982.076050
5,20003,690.176979
7,20004,509.674443
9,20005,541.772269


In [17]:
# Mostrar los mejores modelos del predictor
print("Mejores modelos entrenados:")
print(predictor.leaderboard())

Mejores modelos entrenados:
                       model  score_val  pred_time_val  fit_time_marginal  \
0           WeightedEnsemble  -0.173394       1.677846           0.863149   
1                     DeepAR  -0.181145       0.765715         161.206216   
2  TemporalFusionTransformer  -0.183212       0.569908         373.567895   
3                   PatchTST  -0.183233       0.342223         110.444024   

   fit_order  
0          4  
1          2  
2          1  
3          3  
